In [4]:
!pip install "ale-py == 0.7.4"


In [5]:
!pip install "atari-py == 0.2.9"
!pip install "gym == 0.18.3"
!pip install "tensorflow == 2.5.1"

  Created wheel for gym: filename=gym-0.18.3-py3-none-any.whl size=1657524 sha256=77fd33f9f2f2b020bc7d4fbc14ab94e0a26b2604da9adbf9740fea6db254e690
  Stored in directory: c:\users\rstev\appdata\local\pip\cache\wheels\b3\03\54\9141c232861b89be935b37bdde0ea5ab472f5e18fc20623aed
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.23.0
    Uninstalling gym-0.23.0:
      Successfully uninstalled gym-0.23.0


  Using cached wheel-0.37.1-py2.py3-none-any.whl (35 kB)
  Created wheel for wrapt: filename=wrapt-1.12.1-cp38-cp38-win_amd64.whl size=33696 sha256=bc071b1acc717b5aa7a168e91c6eb65958c8ecd85d8d603b1c05a7c8e606c765
  Stored in directory: c:\users\rstev\appdata\local\pip\cache\wheels\5f\fd\9e\b6cf5890494cb8ef0b5eaff72e5d55a70fb56316007d6dfe73
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4835 sha256=17fc8fa50e7652f6fc794d6d903a30a9f4dd0e86775cea2fb603b5f0061394f8
  Stored in directory: c:\users\rstev\appdata\local\pip\cache\wheels\a0\16\9c\5473df82468f958445479c59e784896fa24f4a5fc024b0f501
Successfully built wrapt termcolor
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.11.2
    Uninstalling wrapt-1.11.2:
      Successfully uninstalled wrapt-1.11.2
  Attempting uninstall: wheel
    Fo

In [25]:
!pip install gym[atari]

  Using cached opencv_python-4.5.5.64-cp36-abi3-win_amd64.whl (35.4 MB)


In [17]:
!pip install keras-rl2

In [14]:
!ls

GC FINAL FINAL.JPG
GC FINAL.JPG
GC.png
Gantt Chart A1.xlsx
Playing Atari with DRL.pdf
ROMS
Skiing-ram-Learning.ipynb


In [1]:
!pip freeze > req.txt

In [3]:
!python -m atari_py.import_roms "./ROMS"

copying adventure.bin from ./ROMS\Adventure (1980) (Atari, Warren Robinett) (CX2613, CX2613P) (PAL).bin to C:\Users\rstev\anaconda3\lib\site-packages\atari_py\atari_roms\adventure.bin
copying air_raid.bin from ./ROMS\Air Raid (Men-A-Vision) (PAL) ~.bin to C:\Users\rstev\anaconda3\lib\site-packages\atari_py\atari_roms\air_raid.bin
copying alien.bin from ./ROMS\Alien (1982) (20th Century Fox Video Games, Douglas 'Dallas North' Neubauer) (11006) ~.bin to C:\Users\rstev\anaconda3\lib\site-packages\atari_py\atari_roms\alien.bin
copying amidar.bin from ./ROMS\Amidar (1982) (Parker Brothers, Ed Temple) (PB5310) ~.bin to C:\Users\rstev\anaconda3\lib\site-packages\atari_py\atari_roms\amidar.bin
copying assault.bin from ./ROMS\Assault (AKA Sky Alien) (1983) (Bomb - Onbase) (CA281).bin to C:\Users\rstev\anaconda3\lib\site-packages\atari_py\atari_roms\assault.bin
copying asterix.bin from ./ROMS\Asterix (AKA Taz) (1984) (Atari, Jerome Domurat, Steve Woita) (CX2696).bin to C:\Users\rstev\anaconda3\l

In [8]:
import gym

env = gym.make("SpaceInvaders-v0")
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [9]:
env.unwrapped.get_action_meanings()


['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [10]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [12]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [21]:
del model

In [22]:
model = build_model(height, width, channels, actions)


In [23]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               34603520  
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_6 (Dense)              (None, 6)                

In [24]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [25]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [26]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))

C:\Users\rstev\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [ ]:
dqn.fit(env, nb_steps=1500, visualize=True, verbose=2)


Training for 1500 steps ...
